## LA's COVID-19 Reopening Indicators 

The NY Times wrote about rushed reopenings being [a global trial-and-error experiment](https://www.nytimes.com/2020/05/07/world/europe/coronavirus-reopening-costs.html) with a further analysis showing how US states reopening [fail to meet the WH guidelines](https://www.nytimes.com/interactive/2020/05/07/us/coronavirus-states-reopen-criteria.html). 

While we wait for LA's own reopening indicators, we can still see how LA stacks up against NY and Chicago's indicators. [NY State's Reopening Indicators](https://www.nytimes.com/2020/05/04/nyregion/coronavirus-reopen-cuomo-ny.html) were based heavily in part by the White House [Opening Up America Again gating criteria](https://www.whitehouse.gov/wp-content/uploads/2020/04/Guidelines-for-Opening-Up-America-Again.pdf), and Chicago outlined even [tougher benchmarks than the IL statewide strategy](https://www.chicagotribune.com/coronavirus/ct-coronavirus-chicago-reopening-lightfoot-20200508-ztpnouwexrcvfdfcr2yccbc53a-story.html). In terms of outbreak severity, [LA and Chicago come right after NYC](https://www.nytimes.com/2020/05/09/us/coronavirus-chicago.html). As of mid May, [LA, DC, and Chicago](https://www.latimes.com/california/story/2020-05-22/white-house-concerned-with-coronavirus-spread-in-l-a-area-asks-cdc-to-investigate) remain hotspots within the US because the caseloads are not consistently dropping.

### [White House Gating Criteria](https://www.whitehouse.gov/wp-content/uploads/2020/04/Guidelines-for-Opening-Up-America-Again.pdf)
#### Symptoms
* Downward trajectory of influenza-like illnesses (ILI) reported within a 14-day period **and**
* Downward trajectory of COVID-like syndromic cases reported within a 14-day period

#### Cases
* Downward trajectory of documented cases within a 14-day period **or**
* Downward trajectory of positive tests as a percent of total tests within a 14-day period (flat or increasing volume of tests) 

#### Hospitals
* Treat all patients without crisis care **and**
* Robust testing program in place for at-risk healthcare workers, including emerging antibody testing

| NY | Chicago |
| :-- | :-- |
| At least 14 days decline in total net hospitalizations on a 3-day rolling average **or** <BR> Less than 15 net new total hospitalizations on a 3-day rolling average </BR> | At least 14 days decline in new cases, on average 
| At least 14 days decline in total net deaths on a 3-day rolling average **or** <BR> Less than 5 new deaths on a 3-day rolling average </BR> | At least 14 days decline in of case rate (per capita?) |
| At least 30% total hospital and ICU beds available  | At least 14 days decline in rate of new cases (per capita?)
| At least 90 days of PPE stockpiled  | Stable or declining rates of hospitalizations  
| Capacity to conduct 30 tests per 1,000 residents per month  | Stable or declining rates of ICU admissions 
| 30 working contact tracers per 100,000 residents | Stable or declining rates of ICU deaths
| Rural regions: Fewer than 2 new COVID patients admitted per 100,000 residents per day | Share of positive results are decreasing below 15% of those tested in the community 
|   | Fewer than 1,800 coronavirus patients in hospital beds 
|   | Fewer than 600 coronavirus patients in ICU beds
|   | Fewer than 450 coronavirus patients on ventilators
|   | Test 135,000 per month, equivalent to 4,500 people per day

In [1]:
import pandas as pd
import pytz

import utils
import meet_indicators
import default_parameters

from datetime import date, datetime, timedelta
from IPython.display import Markdown


# Default parameters
county_state_name = default_parameters.county_state_name
state_name = default_parameters.state_name
msa_name = default_parameters.msa_name
time_zone = default_parameters.time_zone

fulldate_format = default_parameters.fulldate_format
monthdate_format = default_parameters.monthdate_format
start_date = default_parameters.start_date
yesterday_date = default_parameters.yesterday_date
today_date = default_parameters.today_date
two_weeks_ago = default_parameters.two_weeks_ago
two_days_ago = default_parameters.two_days_ago

# Daily testing upper and lower bound
county_test_lower_bound = 10_000
county_test_upper_bound = 16_667
city_test_lower_bound = 5_000
city_test_upper_bound = 8_333
positive_lower_bound = 0.04
positive_upper_bound = 0.10
hospital_bound = 0.30

In [2]:
def summary_of_lacounty_indicators():    
    county_case_indicator = meet_indicators.meet_case("county", county_state_name, start_date)
    county_death_indicator = meet_indicators.meet_death("county", county_state_name, start_date)
    county_test_indicator = meet_indicators.meet_daily_testing(yesterday_date, "county", 
                                                               county_test_lower_bound, county_test_upper_bound)
    county_positive_indicator = meet_indicators.meet_positive_share(yesterday_date, "county", 
                                                                    positive_lower_bound, positive_upper_bound)
    
    # Markdown output
    display(Markdown(f"### Summary of LA County Indicators as of {yesterday_date}:"))
    display(Markdown("#### Cases")) 
    # Cases
    display(Markdown(f"Declining cases: **{county_case_indicator}** out of 14 days"))
    display(Markdown(f"Declining deaths: **{county_death_indicator}** out of 14 days"))
   
    display(Markdown("#### Testing"))
    
    # Daily Testing
    if county_test_indicator >= county_test_upper_bound:
        display(Markdown(f"Daily Testing: <strong><span style='color:blue'>{county_test_indicator:,}</span></strong>"))    
    if (county_test_indicator >= county_test_lower_bound) and (county_test_indicator < county_test_upper_bound):
        display(Markdown(f"Daily Testing: <strong><span style='color:green'>{county_test_indicator:,}</span></strong>"))
    if county_test_indicator < county_test_lower_bound:
        display(Markdown(f"Daily Testing: <strong><span style='color:red'>{county_test_indicator:,}</span></strong>"))          
    
    # Share of Positive Tests
    if county_positive_indicator > positive_upper_bound:
        display(Markdown(f"Share of Positive COVID-19 Results: <strong><span style='color:red'>{county_positive_indicator}</span></strong>"))          
    if (county_positive_indicator >= positive_lower_bound) and (county_positive_indicator < positive_upper_bound):
        display(Markdown(f"Share of Positive COVID-19 Results: <strong><span style='color:green'>{county_positive_indicator}</span></strong>"))          
    if county_positive_indicator < positive_lower_bound:
        display(Markdown(f"Share of Positive COVID-19 Results: <strong><span style='color:blue'>{county_positive_indicator}</span></strong>"))   


In [3]:
def summary_of_lacity_indicators():    
    city_case_indicator = meet_indicators.meet_lacity_case(start_date)
    city_test_indicator = meet_indicators.meet_daily_testing(yesterday_date, "city", 
                                                                   city_test_lower_bound, city_test_upper_bound)
    city_positive_indicator = meet_indicators.meet_positive_share(yesterday_date, "city", 
                                                                  positive_lower_bound, positive_upper_bound)
    acute_indicator = meet_indicators.meet_acute(yesterday_date)
    icu_indicator = meet_indicators.meet_icu(yesterday_date)
    ventilator_indicator = meet_indicators.meet_ventilator(yesterday_date)
    
    # Markdown output
    display(Markdown(f"### Summary of City of LA Indicators as of {yesterday_date}:"))
    display(Markdown("#### Cases")) 
    
    # Cases
    display(Markdown(f"Declining cases: **{city_case_indicator}** out of 14 days"))

    display(Markdown("#### Testing"))
    
    # Daily Testing
    if city_test_indicator >= city_test_upper_bound:
        display(Markdown(f"Daily Testing: <strong><span style='color:blue'>{city_test_indicator:,}</span></strong>"))    
    if (city_test_indicator >= city_test_lower_bound) and (city_test_indicator < city_test_upper_bound):
        display(Markdown(f"Daily Testing: <strong><span style='color:green'>{city_test_indicator:,}</span></strong>"))
    if city_test_indicator < city_test_lower_bound:
        display(Markdown(f"Daily Testing: <strong><span style='color:red'>{city_test_indicator:,}</span></strong>"))          
    
    # Share of Positive Tests
    if city_positive_indicator > positive_upper_bound:
        display(Markdown(f"Share of Positive COVID-19 Results: <strong><span style='color:red'>{city_positive_indicator}</span></strong>"))          
    if (city_positive_indicator >= positive_lower_bound) and (city_positive_indicator < positive_upper_bound):
        display(Markdown(f"Share of Positive COVID-19 Results: <strong><span style='color:green'>{city_positive_indicator}</span></strong>"))          
    if city_positive_indicator < positive_lower_bound:
        display(Markdown(f"Share of Positive COVID-19 Results: <strong><span style='color:blue'>{city_positive_indicator}</span></strong>"))   
        
    display(Markdown("#### Hospital Equipment"))  
    
    # Hospital Equipment
    if acute_indicator >= hospital_bound:
        display(Markdown(f"% Acute Beds: <strong><span style='color:green'>{acute_indicator}</span></strong>"))          
    elif acute_indicator < hospital_bound:
        display(Markdown(f"% Acute Beds: <strong><span style='color:red'>{acute_indicator}</span></strong>"))          

    if icu_indicator >= hospital_bound:
        display(Markdown(f"% ICU Beds: <strong><span style='color:green'>{icu_indicator}</span></strong>"))          
    elif icu_indicator < hospital_bound:
        display(Markdown(f"% ICU Beds: <strong><span style='color:red'>{icu_indicator}</span></strong>")) 
    
    if ventilator_indicator >= hospital_bound:
        display(Markdown(f"% Ventilators: <strong><span style='color:green'>{ventilator_indicator}</span></strong>"))          
    elif ventilator_indicator < hospital_bound:
        display(Markdown(f"% Ventilators: <strong><span style='color:red'>{ventilator_indicator}</span></strong>")) 


In [4]:
summary_of_lacounty_indicators()
summary_of_lacity_indicators()

### Summary of LA County Indicators as of 5/25/20:

#### Cases

Declining cases: **3** out of 14 days

Declining deaths: **10** out of 14 days

#### Testing

...

#### Testing

Share of Positive COVID-19 Results: <strong><span style='color:green'>0.08</span></strong>

#### Hospital Equipment

% Acute Beds: <strong><span style='color:green'>1.0</span></strong>

% Ventilators: <strong><span style='color:green'>1.0</span></strong>

## Caseload Charts

In [5]:
la_county = utils.county_case_charts(county_state_name, start_date)
la_city = utils.lacity_case_charts(start_date) 

alt.HConcatChart(...)

alt.Chart(...)

## Testing Charts

In [6]:
county_tests = utils.lacounty_testing_charts(start_date, county_test_lower_bound, county_test_upper_bound)
positive_tests = utils.lacounty_positive_test_charts(start_date, positive_lower_bound, positive_upper_bound)
city_tests = utils.lacity_testing_charts(start_date, city_test_lower_bound, city_test_upper_bound)
positive_tests = utils.lacity_positive_test_charts(start_date, positive_lower_bound, positive_upper_bound)

alt.LayerChart(...)

alt.LayerChart(...)

alt.Chart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.Chart(...)

## Hospital Capacity Charts

In [7]:
hospital = utils.lacity_hospital_charts(start_date)

alt.LayerChart(...)

alt.Chart(...)